In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import defaultdict, Counter
from itertools import combinations

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [9]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "./words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    ##########################################################
    # my own code #
    ##########################################################
    def cal_percent(self,counter):
        s = float(sum(counter.values()))
        rlist=[]
        for c,count in counter.items():
            rlist.append((c,count/s))
        return rlist
    
    def train_model(self,dic, grams=5):
        tm=defaultdict(Counter)
        td = dic # train data
        for i in range(len(td)-grams):
            gram = td[i:i+grams]
            path = []
            chars = []
            if gram.find(' ') < 0 and gram.find('_') < 0:
                replace = grams-1
                while(replace>0):
                    gram_list = list(gram)
                    comb = combinations(range(len(gram)), replace)
                    # for each gram, calculate the most possible letter
                    for k in list(comb):
                        keep_letter=[]
                        replac_letter=[]
                        k1=list(k)
                        for idx, x in enumerate(gram_list):
                            if idx not in k1:
                                keep_letter.append(x)
                            else:
                                replac_letter.append(x)
                                keep_letter.append('_')
                        path += ["".join(keep_letter)]
                        chars += [Counter("".join(replac_letter)).most_common()[0][0]]
                    replace -= 1
            for history, char in zip(path, chars):
                if history.find(' ') < 0 and char != ' ':
                    tm[history][char]+=1
        
        return_model = {}
        for pa, chars in tm.items():
            return_model[pa]= self.cal_percent(chars)
        
        return return_model

    def check_guessed(self,count_letter, guessed_letters):
        T = False
        for letter, prob in count_letter:
            if letter not in guessed_letters and letter != '_':
                T = True
                break
        if T == True:
            return (letter, prob)
        else:
            return

    
    def guess(self,guessed_letters, word,d, tm,grams=5):
        candidate_list = {} 

        ## first guess
        if len(set(word))== 1 and word[0] == '_':
            count_letter = Counter("".join(d[len(word)])).most_common()
            curr_guess = self.check_guessed(count_letter, guessed_letters)
        
            if curr_guess is not None:
                candidate_list[curr_guess[0]] = curr_guess[1]
        
        ## _pp_e
        for i in range(len(word)):
            gram = word[i:i+grams]
            if gram in tm:
                count_letter = sorted(tm[gram], key=lambda x:x[1], reverse=True)
                curr_guess = self.check_guessed(count_letter, guessed_letters)

                if curr_guess is not None:
                    if curr_guess[0] in candidate_list:
                        candidate_list[curr_guess[0]] += curr_guess[1]
                    else:
                        candidate_list[curr_guess[0]] = curr_guess[1]
                
        if (len(candidate_list) == 0):

            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for guess_letter,instance_count in sorted_letter_count:
                if guess_letter not in self.guessed_letters:
                    letter = guess_letter
                    break

        #     count_letter = Counter("".join(d[len(word)])).most_common()
        #     curr_guess = self.check_guessed(count_letter, guessed_letters)
        
        #     if curr_guess is not None:
        #         if curr_guess[0] in candidate_list:
        #             candidate_list[curr_guess[0]] += curr_guess[1]
        #         else:
        #             candidate_list[curr_guess[0]] = curr_guess[1]

        # if (len(candidate_list) == 0):
        #     letter = random.choice(string.ascii_letters)
        
        else:
            letter = max(candidate_list, key=lambda k: candidate_list[k])

        return letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        tm = self.train_model(" ".join(self.current_dictionary))
        d=defaultdict(list)
        for word in self.current_dictionary:
            d[len(word)].append(word)
                         

        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            g= "_ " * int(len(word)/2)
            guess_clean = g[::2].replace(" ", "")
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                #print("guess_clean: "+guess_clean)
                guess_letter = self.guess(self.guessed_letters, guess_clean,d, tm)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
                    guess_clean = word[::2]
                    #print(word,guess_clean)
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [10]:
api = HangmanAPI(access_token="", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [12]:
# for i in range(5):
#     api.start_game(practice=1,verbose=True)
#     [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
#     practice_success_rate = total_practice_successes / total_practice_runs
#     print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
#     time.sleep(0.5)


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [14]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.35)

Playing  0  th game
Playing  1  th game


KeyboardInterrupt: 

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [15]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.523


Because of the approaching deadline and large CPU usage, I stopped the program at the 433th trail. 